In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# default_exp decoder

In [3]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [4]:
#export
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np # fundamental package for scientific computing with Python
import matplotlib
import matplotlib.pyplot as plt # for plotting

import re

import string
import gc

from fastai2.basics import *
from fastai2.text.all import *
from fastai2.callback.all import *